In [ ]:
# Colab setup code block
try:
    import google.colab
    print("Running on Google Colab. Setting up environment...")
    
    import os
    if not os.path.exists('CMAI-E91-Students'):
        !git clone https://github.com/algolab-quantique/CMAI-E91-Students.git

    os.chdir('CMAI-E91-Students/Part_3_Assignment')
    !pip install -r ../requirements.txt -q
    print("Environment setup complete!")
except ImportError:
    print("Not running on Google Colab. Skipping setup.")


# Défi de programmation - E91 avec |Φ-⟩

Lors de l'atelier, nous avons implémenté le protocole E91 en utilisant l'**état singulet** |Ψ-⟩ = (|01⟩ - |10⟩)/√2.

Dans ce devoir, vous allez adapter l'E91 pour utiliser l'**état de Bell |Φ-⟩** = (|00⟩ - |11⟩)/√2.

**Différence clé :**
- |Ψ-⟩ donne des résultats **anti-corrélés** (Alice=0 → Bob=1)  
- |Φ-⟩ donne des résultats **corrélés** (Alice=0 → Bob=0)

## Ce que vous allez apprendre :

1. **Tout état de Bell fonctionne** pour l'E91, mais il faut utiliser la bonne formule !  
2. Les états **corrélés vs anti-corrélés** influencent l'extraction de la clé  
3. **La formule CHSH change** selon l'état de Bell utilisé  
4. **Approche expérimentale** : essayez toutes les possibilités et trouvez celle qui donne $2\sqrt{2}$

**À FAIRE :** Adaptez le protocole E91 pour utiliser l'état de Bell |Φ-⟩ et déchiffrez les messages secrets.  
- **Tâche 1:** Créez le circuit |Φ-⟩ (astuce : ajoutez UNE porte à |Φ+⟩)  
- **Tâche 2:** Trouvez la formule CHSH correcte (essayez les 4 positions du signe moins !)  


### Configuration

In [ ]:
# ============================================================
# SETUP - All imports and constants
# ============================================================

import random
import numpy as np
from qiskit import QuantumCircuit
import sys
sys.path.append('utils')
import encryption_algorithms as enc

# Import CHSH core functions (solved in Part 1)
from utils.chsh_core import *

# Import E91 helper functions (solved in Part 2)
from utils.e91_core import *

# Use a DIFFERENT seed for this assignment!
GLOBAL_SEED = 42  # Different from workshop (91)
random.seed(GLOBAL_SEED)
np.random.seed(GLOBAL_SEED)

print(" Setup complete!")
print(f"\nAlice's bases: {ALICE_BASES}")
print(f"Bob's bases: {BOB_BASES}")


## Tâche 1 : Créer l'état de Bell |Φ-⟩ (15 points)

Complétez la fonction pour créer |Φ-⟩ = (|00⟩ - |11⟩)/√2

**Astuce :** Commencez par |Φ+⟩ et ajoutez UNE porte pour créer le signe moins !

```
|Φ+⟩: H → CX       gives (|00⟩ + |11⟩)/√2
|Φ-⟩: H → ? → CX   gives (|00⟩ - |11⟩)/√2
```

**Rappel :** La porte Z ajoute une phase π : |1⟩ → -|1⟩


In [ ]:
def create_bell_pair_phi_minus() -> QuantumCircuit:
    """
    TODO: Create the Bell state |Φ-⟩ = (|00⟩ - |11⟩)/√2
    
    Hint: Start with |Φ+⟩ (H + CX) and add ONE gate to create the minus sign!
    """
    qc = QuantumCircuit(2)
    
    # TODO: Add your gates here
    # qc.h(0)     # Hadamard on qubit 0
    # qc.?        # What gate creates the minus sign?
    # qc.cx(0,1)  # CNOT
    
    return qc

In [ ]:
# Test your circuit
phi_minus = create_bell_pair_phi_minus()
print("Your circuit for |Φ-⟩:")
print(phi_minus.draw())

### Vérification de l'état |Φ-⟩

Assurez-vous que vous avez bien créé le bon état de Bell.  
Astuce : $1/\sqrt{2}$ est identique à $\sqrt{2}/2$ 😄


In [ ]:
from qiskit.quantum_info import Statevector
Statevector(phi_minus).draw('latex')

## Fonctions auxiliaires (fournies)

Ce sont les mêmes fonctions que celles de l'atelier. Exécutez cette cellule !


In [ ]:
#  CHSH + E91 helper functions loaded from utils/
# All functions from Part 1 (CHSH) and Part 2 (E91) are ready to use.
print(" Part 1 (CHSH) functions: run_circuit, measure_bell_pair, organize_measurements_by_basis, ...")
print(" Part 2 (E91) functions: generate_random_bases, measure_all_pairs, extract_e91_key_and_bell_test_data, ...")

## Vérification : Testez votre circuit |Φ-⟩

Exécutez ceci pour vérifier si votre circuit est correct. Vous devriez voir environ 50 % de |00⟩ et 50 % de |11⟩.


In [ ]:
# Test the |Φ-⟩ Bell state
phi_minus = create_bell_pair_phi_minus()

# Verify with measurements
test_qc = phi_minus.copy()
test_qc.measure_all()
counts = run_circuit(test_qc, shots=1000)

print(f"\nMeasurement results: {counts}")
print("\n Expected: ~50% |00⟩ and ~50% |11⟩ (never |01⟩ or |10⟩)")
print("   This confirms |Φ-⟩ is CORRELATED : ")
print("   --> Alice and Bob get SAME results: Alice=0 → Bob=0, Alice=1 → Bob=1")

print("\n Note: The minus sign (phase) doesn't affect Z-basis measurements,")
print("   but it DOES affect measurements in rotated bases (used for CHSH)!")

## Tâche 2 : Trouver la bonne formule CHSH

La formule CHSH contient UN signe moins. Pour |Φ-⟩, il se déplace vers une position différente !

**Formule de l'atelier (pour |Ψ-⟩) :**
$$S = E(a,b) - E(a,b') + E(a',b) + E(a',b')$$

**Votre tâche :** Essayez les 4 positions pour le signe moins et trouvez celle qui donne |S| ≈ 2√2 ≈ 2,83


**D’abord, nous générons des données de test pour |Φ⁻⟩ :**

In [ ]:
# Generate test data for |Φ-⟩

print("Generating 500 measurements for CHSH test...\n")
test_num_pairs = 500

# Generate random bases
test_alice_chsh_bases = [random.choice(ALICE_CHSH_BASES) for _ in range(test_num_pairs)]
test_bob_chsh_bases = [random.choice(BOB_CHSH_BASES) for _ in range(test_num_pairs)]

# Generate Bell pairs and Measure them
results = []
for a, b in zip(test_alice_chsh_bases, test_bob_chsh_bases):
    qc = create_bell_pair_phi_minus()
    results.append(measure_bell_pair(qc, a, b))


# step 4: Organize measurements by basis to compute correkations:
measurements = organize_measurements_by_basis(results, test_alice_chsh_bases, test_bob_chsh_bases)
correlations = calculate_correlations(measurements)

# Print correlations
print("Individual correlations E(a,b):")
for pair, E in sorted(correlations.items()):
    print(f"  E{pair} = {E:.4f}")

# Step 5: TODO
# Based on the correlations, compute and find the correct CHSH formulat (in next cell)

In [ ]:
# ═══════════════════════════════════════════════════════════
# TODO: Try all 4 minus sign positions!
# ═══════════════════════════════════════════════════════════

a1, a2 = '0', '90'   # Alice's CHSH bases
b1, b2 = '45', '135' # Bob's CHSH bases

E_ab = correlations[(a1, b1)]              # E(0°, 45°)
E_ab_prime = correlations[(a1, b2)]        # E(0°, 135°)
E_a_prime_b = correlations[(a2, b1)]       # E(90°, 45°)
E_a_prime_b_prime = correlations[(a2, b2)] # E(90°, 135°)

print("="*60)
print("Testing all 4 CHSH formula variations:")
print("="*60)

# TODO: Calculate S for each position of the minus sign

# Position 1: minus at first term
S_minus_p1 = 0  # TODO: -E_ab + E_ab_prime + E_a_prime_b + E_a_prime_b_prime
print(f"\n1. S = -E(a,b) + E(a,b') + E(a',b) + E(a',b')")
print(f"   |S| = {abs(S_minus_p1):.4f}")

# Position 2: minus at second term (original workshop formula)
S_minus_p2 = 0  # TODO: E_ab - E_ab_prime + E_a_prime_b + E_a_prime_b_prime
print(f"\n2. S = +E(a,b) - E(a,b') + E(a',b) + E(a',b')  [workshop formula]")
print(f"   |S| = {abs(S_minus_p2):.4f}")

# Position 3: minus at third term
S_minus_p3 = 0  # TODO: E_ab + E_ab_prime - E_a_prime_b + E_a_prime_b_prime
print(f"\n3. S = +E(a,b) + E(a,b') - E(a',b) + E(a',b')")
print(f"   |S| = {abs(S_minus_p3):.4f}")

# Position 4: minus at fourth term
S_minus_p4 = 0  # TODO: E_ab + E_ab_prime + E_a_prime_b - E_a_prime_b_prime
print(f"\n4. S = +E(a,b) + E(a,b') + E(a',b) - E(a',b')")
print(f"   |S| = {abs(S_minus_p4):.4f}")

print("\n" + "="*60)
print(f"Target: 2√2 ≈ {2*np.sqrt(2):.4f}")
print("The formula with |S| ≈ 2.83 is the CORRECT one!")
print("="*60)

### Votre réponse : Quelle formule CHSH est correcte ?

D'après vos résultats ci-dessus, indiquez la formule CHSH correcte pour |Φ-⟩ :


In [ ]:
def calculate_chsh_value_phi_minus(correlations, alice_bases=ALICE_CHSH_BASES, bob_bases=BOB_CHSH_BASES):
    """
    TODO: Implement the correct CHSH formula for |Φ-⟩
    
    Based on your experiment above, which position gives |S| ≈ 2√2?
    """
    a1, a2 = alice_bases  # '0', '90'
    b1, b2 = bob_bases    # '45', '135'
    
    # TODO: Write the correct formula here
    # S = ???
    S = 0  # Replace with the correct formula!
    
    
    return abs(S)


# Test your formula
chsh = calculate_chsh_value_phi_minus(correlations)
print(f"Your CHSH value: {chsh:.4f}")
print(f"Expected: ≈ {2*np.sqrt(2):.4f}")

## Fonctions auxiliaires du protocole E91

Ces fonctions orchestrent l’ensemble du protocole de distribution de clé E91.


In [ ]:
# ============================================================
# E91 Wrappers — same logic as Part 2, but using YOUR functions
# ============================================================
# These are thin wrappers that call YOUR student-defined functions:
#   - create_bell_pair_phi_minus()        (Task 1)
#   - calculate_chsh_value_phi_minus()    (Task 2)
# All other helpers (measure_all_pairs, extract_e91_key_and_bell_test_data, etc.)
# are already imported from utils/e91_core.py

def create_list_bell_pairs(num_pairs):
    """Create a list of |Φ-⟩ Bell pairs (using YOUR function from Task 1)."""
    return [create_bell_pair_phi_minus() for _ in range(num_pairs)]


def check_for_eavesdropping(chsh_results, chsh_alice_bases, chsh_bob_bases):
    """Run CHSH security test (using YOUR formula from Task 2)."""
    bell_results = organize_measurements_by_basis(chsh_results, chsh_alice_bases, chsh_bob_bases)
    correlations = calculate_correlations(bell_results)
    chsh_value = calculate_chsh_value_phi_minus(correlations)
    return {
        'chsh_value': chsh_value,
        'is_secure': chsh_value > BELL_INEQUALITY_THRESHOLD
    }

print(" E91 wrappers loaded (using your Task 1 & Task 2 functions!)")


## Protocole E91 avec |Φ-⟩ (Fourni)

La fonction `run_e91_protocol` ci-dessous rassemble toutes les étapes.

> **Note :** L'extraction de clé pour |Φ-⟩ est différente de |Ψ-⟩ !
> - |Ψ-⟩ (singulet) est anti-corrélé dans **toutes** les bases → Bob inverse toujours
> - |Φ-⟩ est **corrélé en base Z (0°)** mais **anti-corrélé en base X (90°)**, et non-corrélé à 45°
> - Donc, seules les paires **(90°, 90°)** sont utilisées pour la génération de clé, et Bob inverse ses bits



In [ ]:
# ============================================================
# E91 PROTOCOL with |Φ-⟩ (provided — key extraction differs from |Ψ-⟩)
# ============================================================

# Reset seed for reproducibility
MANUAL_SIMULATOR_SEED_COUNTER = 42
random.seed(GLOBAL_SEED)
np.random.seed(GLOBAL_SEED)

def run_e91_protocol(num_pairs=2000):
    """
    Run E91 protocol with |Φ-⟩ Bell state.

    Key difference from |Ψ-⟩ workshop version:
    - |Φ-⟩ is correlated in Z-basis but ANTI-correlated in X-basis (90°)
    - Only (90°, 90°) pairs are reliable for key generation
    - Bob flips his bits (anti-correlated in X-basis)
    """
    print("=" * 60)
    print("E91 PROTOCOL with |Φ-⟩ Bell State")
    print("=" * 60)

    # Step 1: Generate Bell pairs
    print(f"\n1. Generating {num_pairs} |Φ-⟩ Bell pairs...")
    bell_pairs = create_list_bell_pairs(num_pairs)

    # Step 2: Generate random bases
    print("\n2. Alice and Bob choose random bases...")
    alice_bases = generate_random_bases(num_pairs, ALICE_BASES)
    bob_bases = generate_random_bases(num_pairs, BOB_BASES)

    # Step 3: Measure all pairs
    print("\n3. Measuring all pairs...")
    results = measure_all_pairs(bell_pairs, alice_bases, bob_bases)

    # Step 4: Custom sifting for |Φ-⟩
    print("\n4. Sifting results...")
    key_results = []
    chsh_results = []
    chsh_alice_bases = []
    chsh_bob_bases = []

    for result, a_base, b_base in zip(results, alice_bases, bob_bases):
        if a_base == '90' and b_base == '90':
            # Only 90° pairs for key (anti-correlated, reliable)
            key_results.append(result)
        elif (a_base, b_base) in CHSH_BASIS_PAIRS:
            chsh_results.append(result)
            chsh_alice_bases.append(a_base)
            chsh_bob_bases.append(b_base)
    print(f"   Key generation pairs (90°,90°): {len(key_results)}")
    print(f"   CHSH test pairs: {len(chsh_results)}")

    # Step 5: Security check
    print("\n5. Running CHSH security test...")
    security = check_for_eavesdropping(chsh_results, chsh_alice_bases, chsh_bob_bases)
    print(f"\n   CHSH Value: {security['chsh_value']:.4f}")
    print(f"   Classical limit: 2.0")
    print(f"   Quantum limit: 2√2 ≈ 2.83")

    if not security['is_secure']:
        print("\n SECURITY CHECK FAILED!")
        print("   Possible eavesdropping detected.")
        return None

    print("\n SECURITY CHECK PASSED!")

    # Step 6: Extract key
    print("\n6. Extracting shared key...")
    # |Φ-⟩ at 90° is ANTI-correlated → Bob flips his bits
    # Qiskit format: result = "BA" where r[0]=Bob(q1), r[1]=Alice(q0)
    alice_key = ''.join([r[1] for r in key_results])
    bob_key = ''.join(['1' if r[0] == '0' else '0' for r in key_results])

    if alice_key == bob_key:
        print("\n Keys match perfectly!")
    else:
        mismatches = sum(a != b for a, b in zip(alice_key, bob_key))
        print(f"\n {mismatches} mismatches in {len(alice_key)} bits")

    print(f"   Key length: {len(alice_key)} bits")
    print(f"   Key (first 50): {alice_key[:50]}...")

    return alice_key


print(" E91 Protocol function loaded!")

### Exécuter le protocole E91


In [ ]:
key = run_e91_protocol(num_pairs=800)

### Déchiffrement des messages secrets

Si votre protocole a été exécuté correctement, vous devriez pouvoir déchiffrer les messages !


In [ ]:
def decrypt_and_print_messages(key: str, filename: str = "encrypted_messages.txt"):
    """
    Read encrypted messages from a file, decrypt them using XOR with the given key, and print.

    Args:
        key (str): The key to use for XOR decryption.
        filename (str): The file to read encrypted messages from.
    """
    print("\nDecrypting all messages from", filename)
    with open(filename, "r") as f:
        for line in f:
            if ": " in line:
                msg_id, encrypted = line.split(": ", 1)
                decrypted = enc.decrypt_xor_repeating_key(encrypted.strip(), key)
                print(f"{msg_id}: {decrypted}")


In [ ]:
# For your challenge: must generate the correct key using the E91 protocol to decrypt the messages. Without the key, decryption is not feasible.
# change the path to the encrypted file as needed
path_to_encrypted_file = R"assignment_encrypted_messages.txt"
decrypt_and_print_messages(key, filename=path_to_encrypted_file)


## Résumé : Ce que vous avez appris

Complétez ce tableau en fonction de votre travail :

| Aspect             | Atelier ($\vert \Psi^- \rangle$)           | Ce Devoir ($\vert \Phi^- \rangle$) |
|-------------------|-------------------------------------------|------------------------------------|
| Circuit            | H → CX → X → Z                             | H → ??? → CX                       |
| Corrélation        | Anti-corrélée                              | ???                                |
| Extraction de clé  | Bob inverse les bits                        | ???                                |
| Formule CHSH       | $S = +E(a,b) - E(a,b') + E(a',b) + E(a',b')$ | $S = ???$                          |

**Aperçu clé :** N'importe quel état de Bell peut être utilisé pour l'E91, mais vous devez :  
1. Savoir s'il est **corrélé** ou **anti-corrélé** (pour l'extraction de la clé)  
2. Trouver la **bonne formule CHSH** (celle qui donne $|S| \approx 2\sqrt{2}$)
